# Demo de Ejecución Yolo
**Propuesta para trabajo final, asignatura: SISTEMAS INTELIGENTE, Facultad de Minas, Universidad Nacional de Colombia, Medellín**

A continuación podra ejecutar Yolo (You Look Only Once), en su equipo personal , utilizando la plataforma Colab de google y aprovechando  los servicios de procesamiento e inclusive GPU. Adicionalmente podrá interactuar con su cámara Web, para verificar el reconocmiento de objetos en tiempo real. Tenga en cuenta que el proceso depende altamente de la capacidad de ancho de banda de conexión a Internet. El código está debidamente documentado.

In [ ]:
!git clone https://github.com/...

Cloning into 'YOLO-Object-Detection'...
remote: Enumerating objects: 30, done.
remote: Total 30 (delta 0), reused 0 (delta 0), pack-reused 30
Unpacking objects: 100% (30/30), 12.60 MiB | 13.60 MiB/s, done.


In [ ]:
%cd YOLO-Object-Detection//yolo-object-detection

/content/YOLO-Object-Detection/yolo-object-detection


In [ ]:
!wget "https://pjreddie.com/media/files/yolov3.weights"

--2023-05-24 01:33:13--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M   105MB/s    in 2.2s    

2023-05-24 01:33:15 (105 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [ ]:
from imutils.video import VideoStream
from imutils.video import FPS
import time
import numpy as np
import cv2
import imutils

In [ ]:
yoloPath="yolo-coco"
#yolo --ruta base al directorio YOLO
#confidence --probabilidad mínima de filtrar las detecciones débiles
#threshold --umbral al aplicar supresión no máxima
args={'yolo': yoloPath , 'confidence': 0.5, 'threshold': 0.3}

In [ ]:
# cargar las etiquetas de la clase COCO en las que se entrenó nuestro modelo YOLO
LABELS = open(yoloPath+"//coco.names").read().strip().split("\n")
print("Toal classes {0}".format(len(LABELS)))

Toal classes 80


In [ ]:
# inicializar una lista de colores para representar cada etiqueta de clase posible
np.random.seed(42)
# crear una lista aleatoria de números de tipo entero en el rango 0-255. Tamaño = len (ETIQUETAS), 3... 3 para RGB
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),dtype="uint8")

In [ ]:
# derivar las rutas a los pesos YOLO y configuración del modelo
weightsPath = "yolov3.weights"
configPath = yoloPath+"//yolov3.cfg"
# cargar nuestro detector de objetos YOLO entrenado en el conjunto de datos COCO (80 clases)
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

In [ ]:
#Generalmente en una red secuencial de CNN solo habrá una capa de salida al final.
#En la arquitectura YOLO v3 estamos utilizando múltiples capas de salida que dan predicciones.
ln_all = net.getLayerNames()

ln=[]
# determinar solo los nombres de capa *salida* que necesitamos de YOLO
for i in ln_all:
    if "yolo" in i:
        ln.append(i)

print(ln)

['yolo_82', 'yolo_94', 'yolo_106']


In [ ]:
'''
## Camera Capture
Usar una cámara web para capturar imágenes para procesarlas en tiempo de ejecución.
Fuente: https://colab.research.google.com/notebooks/snippets/advanced_outputs.ipynb#scrollTo=2viqYx97hPMi
'''

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      // mostrar el vídeo en el elemento HTML
      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Cambie el tamaño de la salida para que se ajuste al elemento de vídeo.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // imprime los registros en la celda
      let jsLog = function(abc) {
        document.querySelector("#output-area").appendChild(document.createTextNode(`${abc}... `));
      }

      // Espere a que se haga clic en Capture.
      // Esperar new Promise((resolve) => capture.onclick = resolve);

      for (let i = 0; i < 5; i++) {
        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        img = canvas.toDataURL('image/jpeg', quality);

        // mostrar cada imagen capturada
        jsLog(i + "sending")
        // Llame a una función python y envíe esta imagen
        google.colab.kernel.invokeFunction('notebook.run_algo', [img], {});
        jsLog(i + "SENT")

        // esperar X milisegundos, antes de la próxima captura
        await new Promise(resolve => setTimeout(resolve, 250));
      }

      stream.getVideoTracks()[0].stop(); // stop video stream
    }
    ''')
  display(js) # hacer el HTML proporcionado, parte de la celda
  data = eval_js('takePhoto({})'.format(quality)) # Llamar la función Javascript takePhoto()

In [ ]:
import IPython
from google.colab import output
from google.colab.patches import cv2_imshow

import time
import sys
import numpy as np
import cv2

from PIL import Image
from io import BytesIO
import base64
import logging


def data_uri_to_img(uri):
    """convert base64image to numpy array"""
    try:
        image = base64.b64decode(uri.split(',')[1], validate=True)
        # hacer la imagen binaria, una imagen PIL
        image = Image.open(BytesIO(image))
        # Convertir a un array numpy
        image = np.array(image, dtype=np.uint8);
        return image
    except Exception as e:
        logging.exception(e);
        print('\n')
        return None


def run_algo(imgB64):
    """
    in Colab, run_algo function gets invoked by the JavaScript, that sends N images every second

    params:
      image: image
    """
    image = data_uri_to_img(imgB64)
    frame = image
    if image is None:
        print("At run_algo(): image is None.")
        return

    try:
        # Detección de ejecuciones
        frame = imutils.resize(frame, width=400)
        (H, W) = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
        net.setInput(blob)
        # realizar un pase hacia delante del detector de objetos YOLO, dándonos nuestras cajas delimitadoras y las probabilidades asociadas
        layerOutputs = net.forward(ln)

        # YOLO ahora ha procesado nuestra imagen y obtuvo los datos. ahora solo tenemos que recuperar esos datos
        # inicializar listas vacías
        boxes = []
        confidences = []
        classIDs = []

        # Vamos a empezar a rellenar estas listas con datos de nuestro YOLO layerOutputs
        # Bucle sobre cada una de las salidas de capa
        for output in layerOutputs:
            # bucle sobre cada una de las detecciones
            for detection in output:
                # extraer el ID de clase y la confianza (es decir, probabilidad) de la detección de objetos actuales
                scores = detection[5:]
                classID = np.argmax(scores)
                confidence = scores[classID]

                # filtrar las predicciones débiles asegurando que la probabilidad detectada es mayor que la probabilidad mínima
                if confidence > args["confidence"]:
                    # escalar las coordenadas del cuadro delimitador en relación con el tamaño de la imagen
                    # YOLO devuelve el centro (x, y)-coordenadas del cuadro delimitador seguido de la anchura y altura del cuadro
                    box = detection[0:4] * np.array([W, H, W, H])

                    # convertir valores de caja a valores internos
                    (centerX, centerY, width, height) = box.astype("int")

                    # Usar las coordenadas (x, y)-coordinates para derivar la esquina superior izquierda del cuadro delimitador
                    x = int(centerX - (width / 2))
                    y = int(centerY - (height / 2))

                    # actualizar nuestra lista de coordenadas, confidencias e identificadores de clase
                    boxes.append([x, y, int(width), int(height)])
                    confidences.append(float(confidence))
                    classIDs.append(classID)


            # Aplicar función non-maxima suppression (NMSBoxes()) para suprimir las cajas de límites débiles y superpuestas
            # Todo lo que se requiere es que presentemos los siguientes valores:
            # bounding boxes , confidences , confidence threshold and NMS threshold
        idxs = cv2.dnn.NMSBoxes(boxes, confidences, args["confidence"], args["threshold"])

        # idxs ahora mantenga los índices después de supresión no máxima

        # dibujar los cuadros y el texto de la clase en el marco
        if len(idxs) > 0:
            for i in idxs.flatten():
                # extraer las coordenadas de la caja delimitadora
                (x, y) = (boxes[i][0], boxes[i][1])  # x,y--coordinate of top left corner
                (w, h) = (boxes[i][2], boxes[i][3])

                # Elegir el color
                color = [int(c) for c in COLORS[classIDs[i]]]

                # dibujar un rectángulo de caja delimitadora
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)  # 2 is the line thickness

                # Preparar el texto
                text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])

                # Poner texto en la imagen en x, y-5....un poco arriba de la caja
                cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        cv2_imshow(frame)
    except Exception as e:
        logging.exception(e)

# registrar esta función, por lo que el código JS podría llamar a este
output.register_callback('notebook.run_algo', run_algo)

# poner el código JS en la celda y ejecutarlo
take_photo()
